In [ ]:
!pip install contractions

In [ ]:
import contractions

In [ ]:
import pandas as pd 
import os 
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

import string
from tqdm import tqdm, tqdm_notebook
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import tensorflow_datasets as tfds


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers, losses, metrics, Model
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
from transformers import RobertaTokenizer, RobertaConfig, TFRobertaModel,TFRobertaPreTrainedModel,TFRobertaForSequenceClassification, TFAutoModel, AutoTokenizer
from transformers.modeling_tf_utils import get_initializer

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.tpu.experimental.initialize_tpu_system(resolver)
tpu_strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
class Map(dict):
   
    def __init__(self, *args, **kwargs):
        super(Map, self).__init__(*args, **kwargs)
        for arg in args:
            if isinstance(arg, dict):
                for k, v in arg.iteritems():
                    self[k] = v

        if kwargs:
            for k, v in kwargs.iteritems():
                self[k] = v

    def __getattr__(self, attr):
        return self.get(attr)

    def __setattr__(self, key, value):
        self.__setitem__(key, value)

    def __setitem__(self, key, value):
        super(Map, self).__setitem__(key, value)
        self.__dict__.update({key: value})

    def __delattr__(self, item):
        self.__delitem__(item)

    def __delitem__(self, key):
        super(Map, self).__delitem__(key)
        del self.__dict__[key]

In [ ]:
args = Map()
args.model_name = 'roberta-base'
args.max_len=216
args.batch_size = 8

In [ ]:
df = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
main_df = df.drop(['id','url_legal','license','standard_error'],axis=1)

In [ ]:
def apply_decontraction(text):
    expanded_words = []    
    for word in text.split():      
        expanded_words.append(contractions.fix(word))  
    expanded_text = ' '.join(expanded_words)
    return expanded_text

In [ ]:
translator = str.maketrans('', '', string.punctuation)
def remove_punct(text):
    out = text.translate(translator)
    return out

In [ ]:
main_df['excerpt'] = main_df['excerpt'].apply(lambda x : apply_decontraction(x))
main_df['excerpt'] = main_df['excerpt'].apply(lambda x : remove_punct(x))

In [ ]:
train_df, test_df = train_test_split(main_df, test_size=0.15)
train_df,valid_df = train_test_split(train_df, test_size=0.15)

In [ ]:
train_df.head(5)

In [ ]:
def df_to_dataset(dataframe, shuffle=True):
      dataframe = dataframe.copy()
      labels = dataframe.pop('target')
      ds = tf.data.Dataset.from_tensor_slices((dataframe['excerpt'], labels))
      if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
      #ds = ds.batch(batch_size)
      return ds

In [ ]:
train_ds = df_to_dataset(train_df, shuffle=True)
valid_ds = df_to_dataset(valid_df,shuffle=True)
test_ds = df_to_dataset(test_df,shuffle=True)

In [ ]:
len(test_ds)

In [ ]:
len(train_ds),len(valid_ds),len(test_ds)

In [ ]:
def convert_text_to_feature(text,max_len):
    
      return tokenizer.encode_plus(text, 
                add_special_tokens = False, # add [CLS], [SEP]
                max_length = max_len, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens
                return_attention_mask = True, # add attention mask to not focus on pad tokens
              )

In [ ]:
def map_text_to_dict(input_ids, attention_masks, label):
  return {
      "input_ids": input_ids,      
      "attention_mask": attention_masks,
  }, label

In [ ]:
def encode_text(ds, limit=-1):

  # prepare list, so that we can build up final TensorFlow dataset from slices.
      input_ids_list = []      
      attention_mask_list = []
      label_list = []

      if (limit > 0):
          ds = ds.take(limit)
    
      for text, label in tfds.as_numpy(ds):       

        transformer_input = convert_text_to_feature(text.decode(),args.max_len)
        
        #print(transformer_input)
        input_ids_list.append(transformer_input['input_ids'])        
        attention_mask_list.append(transformer_input['attention_mask'])
        label_list.append(label)

      return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, label_list)).map(map_text_to_dict)


In [ ]:
BATCH_SIZE = args.batch_size * tpu_strategy.num_replicas_in_sync

In [ ]:
#tpu_strategy.num_replicas_in_sync

In [ ]:
ds_train_encoded = encode_text(train_ds).shuffle(100).batch(BATCH_SIZE)
ds_val_encoded = encode_text(valid_ds).batch(BATCH_SIZE)
ds_test_encoded = encode_text(test_ds).batch(BATCH_SIZE)

In [ ]:
##Model Initialization

In [ ]:
# class CustomModel(TFRobertaPreTrainedModel):
#     def __init__(self, config, *inputs, **kwargs):
#         super(CustomModel, self).__init__(config, *inputs, **kwargs)
#         self.num_labels = config.num_labels
#         self.roberta = TFRobertaMainLayer(config, name="roberta")
#         self.dropout_1 = tf.keras.layers.Dropout(0.3)
#         self.classifier = tf.keras.layers.Dense(units=config.num_labels,
#                                                 name='regressor', 
#                                                 kernel_initializer=get_initializer(
#                                                     config.initializer_range))

#     def call(self, inputs, **kwargs):
#         outputs = self.roberta(inputs, **kwargs)
#         pooled_output = outputs[1]
#         pooled_output = self.dropout_1(pooled_output, training=kwargs.get('training', False))
#         logits = self.classifier(pooled_output)
#         outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

#         return outputs

In [ ]:
##predictions ##
test_df = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
config = RobertaConfig(dropout=0.2, attention_dropout=0.2)

In [ ]:
config

In [ ]:
def getCustomModel():
    roberta = 'roberta-base'
    config = RobertaConfig(dropout=0.2, attention_dropout=0.2)
    config.output_hidden_states = False
    transformer_model = TFRobertaModel.from_pretrained(roberta, config = config)

    input_ids_in = tf.keras.layers.Input(shape=(768,), name='input_token', dtype='int32')
    input_masks_in = tf.keras.layers.Input(shape=(768,), name='masked_token', dtype='int32') 

    embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
    X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(embedding_layer)
    X = tf.keras.layers.GlobalMaxPool1D()(X)
    X = tf.keras.layers.Dense(50, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)
    X = tf.keras.layers.Dense(1)(X)
    model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)

    for layer in model.layers[:3]:
      layer.trainable = False
    
    return model

In [ ]:
model = getCustomModel()

In [ ]:
def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

In [ ]:
with tpu_strategy.scope():
    learning_rate = 0.0005
    number_of_epochs=30
    save_best_weights = 'best_weights.h5'
    config = RobertaConfig.from_pretrained(args.model_name, num_labels=1)
    #model = CustomModel.from_pretrained(args.model_name)
    model = TFRobertaForSequenceClassification.from_pretrained(args.model_name, num_labels=1)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
    callbacks = [ModelCheckpoint(save_best_weights, monitor='val_loss', save_best_only=True),
                EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto'),
                ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=3)]
    loss_fn = tf.keras.losses.MeanSquaredError(reduction='sum_over_batch_size')
    model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
roberta_history = model.fit(ds_train_encoded,
                            epochs=number_of_epochs,
                            validation_data=ds_val_encoded,
                            callbacks = callbacks,
                            verbose=1)

In [ ]:
model.summary()

In [ ]:
for record,label in ds_test_encoded:
    

In [ ]:
predictions = model.predict(ds_test_encoded, verbose = True)